In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size())
inp = Variable(torch.randn(1, 1, 32, 32))
out = net(inp)
print(out)

10
torch.Size([6, 1, 5, 5])
Variable containing:
 0.1231 -0.0361  0.1146  0.1438  0.0680 -0.0381  0.0742  0.0098  0.0629 -0.1229
[torch.FloatTensor of size 1x10]



Variable containing:
 0.1325 -0.0250  0.1013  0.1560  0.0711 -0.0584  0.0914  0.0190  0.0421 -0.1492
[torch.FloatTensor of size 1x10]



In [4]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [6]:
output = net(inp)
target = Variable(torch.arange(1, 11)) # a dummy target, for examle
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.2732
[torch.FloatTensor of size 1]



In [7]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [8]:
net.zero_grad() # zeroes the gradient buffers of all parameters
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
 -5.4924
 -2.5059
 -0.5716
  0.3385
 -1.5349
 -2.4974
[torch.FloatTensor of size 6]



In [9]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [11]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [12]:
output = net(inp)
loss = criterion(output, target)
loss.backward()
optimizer.step()